In [2]:
!pip install git+https://github.com/Bots-Avatar/ExplainitAll -q

In [10]:
import os
# os.environ['TEST_MODE_ON_LOW_SPEC_PC'] = 'True'

In [1]:
from explainitall.QA.interp_qa.KNNWithGenerative import FredStruct, PromptBot
from explainitall.QA.extractive_qa_sbert.SVDBert import SVDBertModel
from explainitall.QA.extractive_qa_sbert.QABotsBase import cos_dist
from explainitall.gpt_like_interp.downloader import DownloadManager

from sklearn.neighbors import KNeighborsClassifier
from sentence_transformers import SentenceTransformer
import gensim
from inseq import load_model
from explainitall.gpt_like_interp import interp
from explainitall.gui.interface import set_verbosity_error
set_verbosity_error()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def load_nlp_model(nlp_model_url):
    nlp_model_path = DownloadManager.load_zip(nlp_model_url)
    return gensim.models.KeyedVectors.load_word2vec_format(nlp_model_path, binary=True)

# 'ID': 180
# 'Размер вектора': 300
# 'Корпус': 'Russian National Corpus'
# 'Размер словаря': 189193
# 'Алгоритм': 'Gensim Continuous Bag-of-Words'
# 'Лемматизация': True

nlp_model = load_nlp_model ('http://vectors.nlpl.eu/repository/20/180.zip')

Downloading: /root/.cache/180_zip: 100%|██████████| 462M/462M [00:26<00:00, 18.0MiB/s]   
Extracting: /root/.cache/180_zip_data: 100%|██████████| 4/4 [00:05<00:00,  1.26s/it]


In [4]:
model_path = "sberbank-ai/rugpt3small_based_on_gpt2"

In [5]:
def load_gpt_model(gpt_model_name):
    return load_model(model=gpt_model_name,
                           attribution_method="integrated_gradients")

# 'Фреймворк': 'transformers'
# 'Тренировочные токены': '80 млрд'
# 'Размер контекста': 2048

gpt_model = load_gpt_model(model_path)

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [6]:
import re

def clean_string(text):
    """
    Очистка строки
    """
    seq = text.replace('\n',' ')
    r_char = re.compile('[^A-zА-яЁё0-9": ]')
    r_spaces = re.compile(r"\s+")
    seq = r_char.sub(' ', seq)
    seq = r_spaces.sub(' ', seq).strip()
    return seq.lower()

def value_interp(v):
  if str(v) == 'nan':
    return 'нулевой'
  if v < 0.1:
    return 'незначительной'
  if v < 0.3:
    return 'очень малой'
  if v < 0.45:
    return 'малой'
  if v < 0.65:
    return 'средней'
  if v < 0.85:
    return 'выше средней'
  else:
    return 'очень большой'

def interp_cl(df):
  ret = []
  for index, row in df.iterrows():
    for num_col, col in enumerate(df.columns):
        if num_col != 0:
          value = row[col]

          description = f'Кластер "{row[df.columns[0]]}" влияет на генерацию кластера "{col}" с {value_interp(value)} силой.'
          ret += [description]

  return ret


In [7]:
clusters_discr = [
    {'name': 'Животные', 'centroid': ['собака', 'кошка', 'заяц'], 'top_k': 140},
    {'name': 'Лекарства', 'centroid': ['уколы', 'таблетки', 'микстуры'], 'top_k': 160},
    {'name': 'Болезни', 'centroid': ['простуда', 'орви', 'орз', 'грипп'], 'top_k': 20},
    {'name': 'Аллергия', 'centroid': ['аллергия'], 'top_k': 20}
]

explainer = interp.ExplainerGPT2(gpt_model=gpt_model, nlp_model=nlp_model)


input_text = 'у кошки грипп и аллергия на антибиотбиотики вопрос: чем лечить кошку? ответ:'
generated_text = 'лечите ее уколами'

expl_data = explainer.interpret(
    input_texts=input_text,
    generated_texts=generated_text,
    clusters_description=clusters_discr,
    batch_size=50,
    steps=34,
)

Attributing with integrated_gradients...: 100%|██████████| 26/26 [00:01<00:00,  3.64it/s]


In [8]:
# Результат интерпретации
imp_df_cl = expl_data.cluster_imp_aggr_df
cl_desc = interp_cl(imp_df_cl)

In [11]:
path_sbert = 'FractalGPT/SbertSVDDistil'
sbert = SentenceTransformer(path_sbert)
sbert[0].auto_model = SVDBertModel.from_pretrained(path_sbert)

if os.getenv('TEST_MODE_ON_LOW_SPEC_PC') == 'True':
    fred = FredStruct('t5-small' )
else:
    fred = FredStruct('FractalGPT/FRED-T5-Interp')

tokenizer_config.json:   0%|          | 0.00/20.1k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.76M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.74k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [12]:
cl_desc

['Кластер "Аллергия" влияет на генерацию кластера "Лекарства" с выше средней силой.',
 'Кластер "Болезни" влияет на генерацию кластера "Лекарства" с выше средней силой.',
 'Кластер "Животные" влияет на генерацию кластера "Лекарства" с средней силой.',
 'Кластер "Лекарства" влияет на генерацию кластера "Лекарства" с нулевой силой.']

In [13]:
clean = [clean_string(cl_data) for cl_data in cl_desc]
vects_x = sbert.encode(clean)
m = vects_x.mean(axis=0)
s = vects_x.std(axis=0)
knn_vects_x = (vects_x - m)/s
knn = KNeighborsClassifier(metric=cos_dist)
knn.fit(knn_vects_x, cl_desc)

interp_bot = PromptBot(knn, sbert, fred, cl_desc, device='cpu')

In [14]:
ans = interp_bot.get_answers('Как влияет аллергия на назначение лекарства', top_k=3)
ans.split('.')[0]

'Кластер "Аллергия" влияет на генерацию кластера "Лекарства" с выше средней силой'

In [15]:
ans = interp_bot.get_answers('Как влияет кластер болезни на кластер лекарства', top_k=3)
ans.split('.')[0]

'Кластер "Болезни" влияет на кластер "Лекарства" с выше средней силой Кластер "Аллергия" влияет кластером "Лекаря" с нулевой силой'